In [1]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch

import numpy as np
import cv2

import os  
from pathlib import Path
import sys

import matplotlib.pyplot as plt

from einops import rearrange

import glob
from sklearn.model_selection import train_test_split

In [2]:
path = Path(os.getcwd())
parent = path.parent
parent = str(parent)
path = str(path)
print(path)
sys.path.insert(1, path)
sys.path.insert(1, parent)

/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio


In [3]:
from dataloader.dataset_generator_class import ImageDatasetGenerator
from models.CvT_model import CvT
from angular_dist_loss.angular_dist_loss import CosineDistanceMetric

In [4]:
import torch
import torch.nn.functional as F

# Create dummy image data with 3 color channels (RGB) and spatial dimensions 10x10
height, width, channels = 10, 10, 3
image1 = torch.randn(1, channels, height, width)  # Batch size = 1
image2 = torch.randn(1, channels, height, width)  # Batch size = 1

# Compute cosine similarity along the channel dimension (dim=1)
cosine_similarity_map = F.cosine_similarity(image1, image2, dim=1)  # Shape: (1, height, width)

# Inspect the result
print("Cosine Similarity Map Shape:", cosine_similarity_map.shape)
print("Cosine Similarity Map (Sample):", cosine_similarity_map)


Cosine Similarity Map Shape: torch.Size([1, 10, 10])
Cosine Similarity Map (Sample): tensor([[[ 0.9220, -0.5231,  0.4831, -0.6378, -0.8759,  0.4608,  0.2581,
          -0.8016, -0.5303,  0.9791],
         [-0.3771,  0.7605, -0.5386,  0.4234,  0.9876, -0.7410, -0.0201,
           0.0363,  0.6981, -0.7820],
         [-0.4257, -0.1823,  0.6723, -0.3497, -0.4020,  0.0639, -0.0143,
           0.1009,  0.9442,  0.8958],
         [ 0.7189, -0.6682,  0.7644,  0.3612, -0.3193, -0.5670, -0.7822,
          -0.4440,  0.8474,  0.1170],
         [ 0.6589, -0.2912, -0.4562, -0.6379,  0.8324,  0.0086,  0.2314,
           0.9306, -0.5040,  0.5117],
         [ 0.6265,  0.7784, -0.9426, -0.1547, -0.1517, -0.6287,  0.7963,
           0.3490, -0.7385,  0.3366],
         [ 0.3273,  0.7947, -0.5527,  0.5817, -0.6719, -0.0039,  0.8875,
           0.2409,  0.0701, -0.8399],
         [-0.6304, -0.1355,  0.3797, -0.7333, -0.6063, -0.7245, -0.4952,
          -0.2359,  0.9530, -0.0600],
         [ 0.0460,  0.8652,

In [4]:
# Transforms -- We can add augmentations here
transform_images = transforms.Compose([ 
                        transforms.ToTensor()       
                        #transforms.Normalize(mean = mean, std = std)
                                    ])

transform_guidance = transforms.Compose([ 
                        transforms.ToTensor()
                                    ])

In [5]:
# Folder paths
image_folder = "/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio/training_data/training_images_cropped"
isd_map_folder = "/Users/nelsonfarrell/Documents/Northeastern/7180/projects/spectral_ratio/training_data/training_isds_cropped"

# Create dataset
dataset = ImageDatasetGenerator(image_folder, 
                                isd_map_folder, 
                                split = None, 
                                val_size = 0.2, 
                                random_seed = 42, 
                                transform_images = transform_images, 
                                transform_guidance = transform_guidance)

# Create dataset
train_dataset = ImageDatasetGenerator(image_folder, 
                                      isd_map_folder, 
                                      split = "train", 
                                      val_size = 0.2, 
                                      random_seed = 42, 
                                      transform_images = transform_images, 
                                      transform_guidance = transform_guidance)

# Create dataset
val_dataset = ImageDatasetGenerator(image_folder, 
                                    isd_map_folder, 
                                    split = "val", 
                                    val_size = 0.2, 
                                    random_seed = 42, 
                                    transform_images = transform_images, 
                                    transform_guidance = transform_guidance)

# Create dataloader
# Shuffle equals false for now for testing
# drop last creates only full batches
dataloader = DataLoader(val_dataset, batch_size = 4, shuffle = False, drop_last = True)

Assertion Passed!!! We have the same number of images and ISD maps.
Assertion Passed!!! We have the same number of images and ISD maps.
Assertion Passed!!! We have the same number of images and ISD maps.


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'  
embed_size = 64
num_class = 10
model = CvT(embed_size).to(device)

Entering embedding: Patch Size: 7 -- Stride: 4 -- Embedding Dims 64
Entering embedding: Patch Size: 3 -- Stride: 2 -- Embedding Dims 192
Entering embedding: Patch Size: 3 -- Stride: 2 -- Embedding Dims 384


In [7]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 10
learning_rate = 0.001
batch_size = 4

# Define loss function and optimizer
criterion = CosineDistanceMetric()  # Assuming regression; adjust for classification if needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()
    train_loss = 0.0

    # Training Step
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    for batch in tqdm(train_loader, desc="Training"):
        images, guidance_maps = batch
        images, guidance_maps = images.to(device), guidance_maps.to(device)

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, guidance_maps)
        print("Loss:", loss.item())

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation Step
    model.eval()
    val_loss = 0.0
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            images, guidance_maps = batch
            images, guidance_maps = images.to(device), guidance_maps.to(device)

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = criterion(outputs, guidance_maps)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}")

print("Training Complete!")


Epoch 1/10


Training:   0%|          | 0/298 [00:00<?, ?it/s]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 1.0288641452789307


Training:   0%|          | 1/298 [00:01<06:34,  1.33s/it]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 0.8943011164665222


...

Training:   3%|▎         | 8/298 [00:09<05:25,  1.12s/it]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 1.2715801000595093


Training:   3%|▎         | 9/298 [00:10<05:22,  1.12s/it]

Stage 1: Shape x: torch.Size([4, 64, 55, 55])
Stage 2: Shape x: torch.Size([4, 192, 27, 27])
Stage 3: Shape x: torch.Size([4, 384, 13, 13])
Loss: 0.9528747797012329


Training:   3%|▎         | 10/298 [00:11<05:21,  1.12s/it]